In [ ]:
import polars as pl
import pandas as pd

In [ ]:
df = pl.read_csv("Data/Split1/*.csv").to_pandas()

In [4]:
df.columns

Index(['text', 'source', 'prompt_id', 'text_length', 'word_count', 'label',
       'gpt2-large-text_ppl', 'gpt2-large-max_sent_ppl',
       'gpt2-large-sent_ppl_avg', 'gpt2-large-sent_ppl_std',
       'gpt2-large-max_step_ppl', 'gpt2-large-step_ppl_avg',
       'gpt2-large-step_ppl_std', 'gpt2-large-rank_0', 'gpt2-large-rank_10',
       'gpt2-large-rank_100', 'gpt2-large-rank_1000'],
      dtype='object')

In [5]:
x = df[['gpt2-large-text_ppl', 'gpt2-large-max_sent_ppl',
       'gpt2-large-sent_ppl_avg', 'gpt2-large-sent_ppl_std',
       'gpt2-large-max_step_ppl', 'gpt2-large-step_ppl_avg',
       'gpt2-large-step_ppl_std', 'gpt2-large-rank_0', 'gpt2-large-rank_10',
       'gpt2-large-rank_100', 'gpt2-large-rank_1000']]

y = df[['label']]

In [9]:
from sklearn.ensemble import RandomForestClassifier

In [10]:
rf= RandomForestClassifier()

rf.fit(x,y)

/Users/t-andrew.widjaya/Documents/Thesis/.venv/lib/python3.9/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier()

In [11]:
import pickle

In [13]:
import pickle
with open('Model.pkl', 'wb') as fp:
    pickle.dump(rf, fp)

In [26]:
Text = """[Your Name]
[Your Address]
[City, State, ZIP]
[Email Address]
[Phone Number]
[Date]

[Senator's Name]
[Senator's Address]
[City, State, ZIP]

Subject: The Importance of Keeping the Electoral College

Dear Senator [Senator's Last Name],

I am writing to express my support for keeping the Electoral College as the method for electing the President of the United States. While there have been calls to change to a popular vote system, I believe that the Electoral College serves an essential role in maintaining the integrity and representation of our nation's highest office.

One of the fundamental reasons for maintaining the Electoral College is the certainty of outcome it provides. Unlike a direct popular vote, the Electoral College ensures that the winning candidate's share of the electoral votes typically exceeds their share of the popular vote. This system reduces the likelihood of disputes over the outcome, as seen in the 2000 election. The winner-take-all method of awarding electoral votes also prevents a tie in the nationwide electoral vote, which would create significant uncertainty and potential chaos in the election process.

Moreover, the Electoral College system ensures that the President is everyone's President. As you may know, no single region in the country has enough electoral votes to elect a President. This requirement compels presidential candidates to have trans-regional appeal, campaigning across various states and engaging with diverse constituencies. Consequently, this promotes a sense of representation among the residents of different regions, preventing the perception that their votes do not count or that the President does not have their interests at heart.

I also believe that swing states play a crucial role in the Electoral College system. The winner-take-all method, employed in most states, incentivizes candidates to focus their campaign efforts on swing states. This results in increased attention, detailed policy discussions, and targeted outreach to voters in those states. Swing states tend to have considered and engaged voters who are well-informed about the candidates' positions. By allowing these states to have a significant influence on the outcome, we ensure that the most thoughtful voters are the ones who ultimately decide the election.

Furthermore, the Electoral College provides a balance of power between large and small states. The system gives more weight to larger states by granting them more electoral votes based on their population size. Without the Electoral College, smaller states may feel neglected, as candidates would be less likely to campaign in those regions. Maintaining a system that rewards states of all sizes encourages candidates to pay attention to a broader range of voters and ensures that the President is accountable to the entire nation.

Lastly, the Electoral College eliminates the need for run-off elections. In instances where no candidate receives a majority of the votes, the Electoral College ensures a clear winner. This reduces the complexities and costs associated with organizing run-off elections, thus streamlining the process and allowing for a swift and decisive outcome.

In conclusion, the Electoral College is essential for upholding the principles of our democracy. It provides certainty, encourages trans-regional appeal, promotes engagement in swing states, balances the power between large and small states, and eliminates the need for run-off elections. While some may argue that a popular vote would be more democratic, the Electoral College ensures a more representative outcome that takes into account the unique characteristics of our nation.

Thank you for considering my perspective on this matter. I hope you will continue to support the Electoral College as an integral part of our democratic process.

Sincerely,

[Your Name]"""

In [18]:
import torch
from nltk.tokenize import PunktSentenceTokenizer
CROSS_ENTROPY = torch.nn.CrossEntropyLoss(reduction='none')
NLTK          = PunktSentenceTokenizer()
DEVICE        = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# DEVICE        = torch.device('cpu')

In [19]:
sent_cut_en = NLTK.tokenize

In [20]:
def gpt2_features(text, tokenizer, model, sent_cut):
    # Tokenize
    input_max_length = tokenizer.model_max_length - 2
    token_ids, offsets = list(), list()
    sentences = sent_cut(text)
    
    for s in sentences:
        tokens = tokenizer.tokenize(s)
        ids = tokenizer.convert_tokens_to_ids(tokens)
        difference = len(token_ids) + len(ids) - input_max_length
        if difference > 0:
            ids = ids[:-difference]
        offsets.append((len(token_ids), len(token_ids) + len(ids)))
        token_ids.extend(ids)
        if difference >= 0:
            break

    input_ids = torch.tensor([tokenizer.bos_token_id] + token_ids).to(DEVICE)
    logits = model(input_ids).logits
    
    # Shift so that n-1 predict n
    shift_logits = logits[:-1].contiguous()
    shift_target = input_ids[1:].contiguous()
    loss = CROSS_ENTROPY(shift_logits, shift_target)

    all_probs = torch.softmax(shift_logits, dim=-1)
    sorted_ids = torch.argsort(all_probs, dim=-1, descending=True)  # stable=True
    expanded_tokens = shift_target.unsqueeze(-1).expand_as(sorted_ids)
    indices = torch.where(sorted_ids == expanded_tokens)
    rank = indices[-1]
    counter = [
        rank < 10,
        (rank >= 10) & (rank < 100),
        (rank >= 100) & (rank < 1000),
        rank >= 1000
    ]
    counter = [c.long().sum(-1).item() for c in counter]


    # compute different-level ppl
    text_ppl = loss.mean().exp().item()
    sent_ppl = list()
    for start, end in offsets:
        nll = loss[start: end].sum() / (end - start)
        sent_ppl.append(nll.exp().item())
        
    max_sent_ppl = max(sent_ppl)
    sent_ppl_avg = sum(sent_ppl) / len(sent_ppl)
    if len(sent_ppl) > 1:
        sent_ppl_std = torch.std(torch.tensor(sent_ppl)).item()
    else:
        sent_ppl_std = 0

    mask = torch.tensor([1] * loss.size(0)).to(DEVICE)
    step_ppl = loss.cumsum(dim=-1).div(mask.cumsum(dim=-1)).exp()
    max_step_ppl = step_ppl.max(dim=-1)[0].item()
    step_ppl_avg = step_ppl.sum(dim=-1).div(loss.size(0)).item()
    if step_ppl.size(0) > 1:
        step_ppl_std = step_ppl.std().item()
    else:
        step_ppl_std = 0
    ppls = [
        text_ppl, max_sent_ppl, sent_ppl_avg, sent_ppl_std,
        max_step_ppl, step_ppl_avg, step_ppl_std
    ]
    return counter, ppls  # type: ignore

In [22]:
cols = [
    'text_ppl', 'max_sent_ppl', 'sent_ppl_avg', 'sent_ppl_std', 'max_step_ppl', 
    'step_ppl_avg', 'step_ppl_std', 'rank_0', 'rank_10', 'rank_100', 'rank_1000'
]

final_cols = [
    'text',  
    'source',
    'prompt_id',
    'text_length' ,
    'word_count',
    'label', 'text_ppl', 'max_sent_ppl', 'sent_ppl_avg', 'sent_ppl_std', 'max_step_ppl', 
    'step_ppl_avg', 'step_ppl_std', 'rank_0', 'rank_10', 'rank_100', 'rank_1000'
]

In [28]:
from tqdm.auto import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelWithLMHead, AutoModelForCausalLM
import numpy as np

In [29]:
model_train_feats = []
for model in tqdm(['gpt2-large']):
        TOKENIZER_EN = AutoTokenizer.from_pretrained(f'{model}/tokenizer')
        MODEL_EN     = AutoModelForCausalLM.from_pretrained(f'{model}/model').to(DEVICE)
        
        train_ppl_feats  = []
        train_gltr_feats = []
        with torch.no_grad():
                gltr, ppl = gpt2_features(Text, TOKENIZER_EN, MODEL_EN, sent_cut_en)
                train_ppl_feats.append(ppl)
                train_gltr_feats.append(gltr)
                
        X_train = pd.DataFrame(
            np.concatenate((train_ppl_feats, train_gltr_feats), axis=1), 
            columns=[f'{model}-{col}' for col in cols]
        )
        model_train_feats.append(X_train)
        

100%|██████████| 1/1 [00:13<00:00, 13.87s/it]


In [30]:
model_train_feats

[   gpt2-large-text_ppl  gpt2-large-max_sent_ppl  gpt2-large-sent_ppl_avg  \
 0             7.635703                32.642498                 8.721887   
 
    gpt2-large-sent_ppl_std  gpt2-large-max_step_ppl  gpt2-large-step_ppl_avg  \
 0                 5.614326              1083.938599                11.132915   
 
    gpt2-large-step_ppl_std  gpt2-large-rank_0  gpt2-large-rank_10  \
 0                  45.7388              626.0                63.0   
 
    gpt2-large-rank_100  gpt2-large-rank_1000  
 0                 25.0                   5.0  ]

In [33]:
train_feats = pd.concat(model_train_feats, axis=1)
train_feats

,gpt2-large-text_ppl,gpt2-large-max_sent_ppl,gpt2-large-sent_ppl_avg,gpt2-large-sent_ppl_std,gpt2-large-max_step_ppl,gpt2-large-step_ppl_avg,gpt2-large-step_ppl_std,gpt2-large-rank_0,gpt2-large-rank_10,gpt2-large-rank_100,gpt2-large-rank_1000
0,7.635703,32.642498,8.721887,5.614326,1083.938599,11.132915,45.7388,626.0,63.0,25.0,5.0


In [36]:
with open('Model.pkl', 'rb') as fp:
    rf_bayangan = pickle.load(fp)

In [37]:
rf_bayangan.predict_proba(train_feats)

array([[0., 1.]])